# Поиск похожих изображений
## Черпал идеи:
###### https://habr.com/ru/post/206264/
###### https://habr.com/ru/post/120562/
###### https://medium.com/@senior_sigan/similar-images-search-ce433491059b#3344
###### https://youtu.be/3nct0Kalk2I?t=4108
###### https://ru.wikipedia.org/wiki/SSIM
###### https://www.phash.org/
###### https://habr.com/ru/post/198338/
###### https://pillow.readthedocs.io/en/stable/reference/Image.html

In [3]:
import os
import numpy as np
from PIL import Image, ImageFilter

## pHash 
для поиска модифицированных изображений

In [81]:
def main_p(name1):
    img1 = np.array(np.asarray(Image.open(f'{name1}').resize((32,32), Image.ANTIALIAS).convert('L')),dtype=np.double)
    dct_matrix = np.zeros((32,32))
    for i in range(32):
        for j in range(32):
            ij = 0
            for n1 in range(32):
                temp = 0
                for n2 in range(32):
                    temp += img1[n1][n2]*np.cos(np.pi * (n1 + 0.5) * i / 32) * np.cos(np.pi * (n2 + 0.5) * j / 32)
                ij += temp
            dct_matrix[i][j] = ij
            
    res = dct_matrix[:8, :8]
    res_c = np.copy(res)
    res_c[0][0] = 0
    mean_res = res_c.mean()
    
    return (np.where(res < mean_res, 0, 1)).reshape((64,))

In [82]:
def hamm(a,b):
    return np.count_nonzero(a != b)

In [83]:
def pHash(path = './dev_dataset'):
    list_of_images = os.listdir(path)
    k = len(list_of_images)
    res = np.zeros((k,k))
    hash_of_images = []
    
    for i in list_of_images:
        hash_of_images.append(main_p(path+'/'+i))
    
    for i in range(k):
        for j in range(i+1,k):
            res[i][j] = hamm(hash_of_images[i], hash_of_images[j])

    return  hash_of_images

## Хэш-функция по среднему 
для поиска похожих изображений (смещение точки обзора)

In [7]:
def creator(path_of_image):
    image1 = np.array(np.asarray(Image.open(f'{path_of_image}').resize((10, 10), Image.ANTIALIAS).convert('L')),dtype=np.double)
    return np.where(image1 > image1.mean(), 1, 0)

In [8]:
def hamm(ar1, ar2):
    return np.count_nonzero(ar1.reshape((64,)) != ar2.reshape((64,)))

In [9]:
def eq(ar1, ar2, h):
    return hamm(ar1, ar2) <= h

In [10]:
def shift(ar1, ar2, h):
    
    def equality(ar1, ar2):
        return eq(ar1, ar2, h)
        
    return  equality(ar1, ar2[0:8,  0:8])  or \
            equality(ar1, ar2[0:8,  1:9])  or \
            equality(ar1, ar2[2:10, 1:9])  or \
            equality(ar1, ar2[2:10, 2:10]) or \
            equality(ar1, ar2[0:8,  2:10]) or \
            equality(ar1, ar2[2:10, 0:8])  or \
            equality(ar1, ar2[1:9,  0:8])  or \
            equality(ar1, ar2[1:9,  2:10])

In [11]:
def is_equal_with_shift(img1, img2, h):
    centr1 = img1[1:9, 1:9]
    centr2 = img2[1:9, 1:9]
    
    return shift(centr1, img2, h) or shift(centr2, img1, h)

In [12]:
def simple_hash(path = './dev_dataset'):
    list_of_images = os.listdir(path)
    k = len(list_of_images)
    hash_of_images = []
    
    for i in list_of_images:
        hash_of_images.append(creator(path+'/'+i))
        
    return hash_of_images

def compare_simple_hashes(hash_of_images):
    h = 11
    for i in range(k):
        for j in range(i+1,k):
            res[i][j] = is_equal_with_shift(hash_of_images[i], hash_of_images[j], h)
    return res

## Обьеденяем
phash - 21
just_hash - 11

In [105]:
path = './dev_dataset'
list_of_images = os.listdir(path)
k = len(list_of_images)
matrix_to_train_2 = np.zeros((int(k*(k-1)/2), 3))
simple_hash_images = simple_hash(path)
phash_of_images = pHash(path)

In [106]:
f = 0
list_of_images = os.listdir(path)
k = len(list_of_images)
for i in range(k):
    for j in range(i+1,k):
        matrix_to_train_2[f][0] = hamm(phash_of_images[i], phash_of_images[j])
        matrix_to_train_2[f][1] = is_equal_with_shift(simple_hash_images[i], simple_hash_images[j], 11)
        matrix_to_train_2[f][2] = list_of_images[i][:2] == list_of_images[j][:2]
        if list_of_images[i][:2] == list_of_images[j][:2]:
            print(matrix_to_train_2[f][0], f, i,j, '-----', list_of_images[i], list_of_images[j])
        f += 1

15.0 48 3 4 ----- 04.jpg 04_s.jpg
10.0 75 5 6 ----- 06.jpg 06_s.jpg
0.0 117 9 10 ----- 11.jpg 11_duplicate.jpg
0.0 118 9 11 ----- 11.jpg 11_modification.jpg
0.0 125 10 11 ----- 11_duplicate.jpg 11_modification.jpg
1.0 150 15 16 ----- 15.jpg 15_modification.jpg


In [109]:
df_2 = pd.DataFrame(data=matrix_to_train_2, columns=['1', '2', '3'])
aa = df_2[df_2['3'] == 1 ]
aa

,1,2,3
48,15.0,1.0,1.0
75,10.0,1.0,1.0
117,0.0,0.0,1.0
118,0.0,0.0,1.0
125,0.0,0.0,1.0
150,1.0,1.0,1.0


In [65]:
import pandas as pd
df = pd.DataFrame(data=matrix_to_train, columns=['1', '2', '3'])
aa = df[df['1'] > 21.0 ]
to_check = aa[aa['2'] == 1.0]

In [54]:
bad_indexs = np.array(aa[aa['2'] == 1.0].index)

In [57]:
150 in bad_indexs

False

In [60]:
f = 0
pleasue = []
for i in range(k):
    for j in range(i+1,k):
        if f in bad_indexs:
            pleasue.append((i,j))
        f += 1

In [73]:
def ssim(img1, img2):

    K = [0.01, 0.03]
    L = 32767 #16777215          # 2 ** bpp(img1) - 1

    C1 = (K[0] * L) ** 2
    C2 = (K[1] * L) ** 2

    mu1 = np.mean(img1)
    mu2 = np.mean(img2)

    mu1_sq = np.multiply(mu1, mu1)
    mu2_sq = np.multiply(mu2, mu2)
    mu1_mu2 = np.multiply(mu1, mu2)

    sigma1_sq = np.cov(img1,img1) ** 2
    sigma2_sq = np.cov(img1,img1) ** 2
    sigma12 = np.cov(img1,img2) ** 2

    ssim_map = np.divide(np.multiply((2*mu1_mu2 + C1), (2*sigma12 + C2)), np.multiply((mu1_sq + mu2_sq + C1),(sigma1_sq + sigma2_sq + C2)))
    return np.mean(ssim_map)

def main(name1, name2):

    img1 = np.array(np.asarray(Image.open(f'{name1}').resize((1024,1024), Image.ANTIALIAS).convert(mode = 'L')),dtype=np.double)
    img2 = np.array(np.asarray(Image.open(f'{name2}').resize((1024,1024), Image.ANTIALIAS).convert(mode = 'L')),dtype=np.double)

    return ssim(img1, img2)

In [76]:
ssim_hashes = np.zeros((len(pleasue),))
k=0
for i,j in pleasue:
    ssim_hashes[k] = abs(main(path+'/'+list_of_images[i], path+'/'+list_of_images[j]) - 1)
    print(list_of_images[i], list_of_images[j], ssim_hashes[k])
    k += 1

02-mod-1.jpg 10-mod-2.jpg 0.09586846117432879
02-mod-1.jpg 10.jpg 0.11480217712228646
02-mod-1.jpg 14.jpg 0.5461573666921056
02-mod-1.jpg 23_s.JPG 1.0718805748573774
02.jpg 10-mod-2.jpg 0.09802487258861914
02.jpg 10.jpg 0.11829893593517538
02.jpg 14.jpg 0.5221291302310045
02.jpg 23_s.JPG 1.0440694885824593
03-mod-2.jpg 06.jpg 0.0877786633043609
03-mod-2.jpg 14-mod-2.jpg 0.18634158599677786
03.jpg 06.jpg 0.08679800603193
03.jpg 14-mod-2.jpg 0.18434329954316597
06.jpg 14-mod-2.jpg 0.4168588887490434
06.jpg 14.jpg 0.2874981110708814
06_s.jpg 14-mod-2.jpg 0.4314431269674217
06_s.jpg 14.jpg 0.3021381304949535
10-mod-2.jpg 17-mod-2.jpg 1.5947411563833969
10-mod-2.jpg 23_s.JPG 0.7750384375307604
10.jpg 17-mod-2.jpg 1.6242521179486697
10.jpg 23_s.JPG 0.790519739011301
16-mod-2.jpg 21_s.JPG 1.3543431014640395
16-mod-2.jpg 25.JPG 0.025054262828891005
16-mod-2.jpg 25_s.JPG 0.7464358550848706
16.jpg 21_s.JPG 1.3827734352726302
16.jpg 25.JPG 0.020475204098244326
16.jpg 25_s.JPG 0.746359198248169
17

In [75]:
ssim_hashes

array([0.09586846, 0.11480218, 0.54615737, 1.07188057, 0.09802487,
       0.11829894, 0.52212913, 1.04406949, 0.08777866, 0.18634159,
       0.08679801, 0.1843433 , 0.41685889, 0.28749811, 0.43144313,
       0.30213813, 1.59474116, 0.77503844, 1.62425212, 0.79051974,
       1.3543431 , 0.02505426, 0.74643586, 1.38277344, 0.0204752 ,
       0.7463592 , 1.23342698, 1.22952069, 0.93502459, 0.47134944,
       0.49239341, 0.47866284, 0.47651595, 0.52065642, 0.49220328,
       0.49281166, 0.20986531, 0.24989504, 0.02731359, 0.86047593,
       1.34417414, 0.516876  ])

In [67]:
to_check['0'] = ssim_hashes
to_check

C:\Users\milan\Anaconda3\envs\ipykernel_py3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,1,2,3,0
127,33.0,1.0,0.0,0.161396
128,33.0,1.0,0.0,0.163964
137,30.0,1.0,0.0,0.002112
162,39.0,1.0,0.0,0.166881
183,33.0,1.0,0.0,0.162830
184,33.0,1.0,0.0,0.165560
193,30.0,1.0,0.0,0.001802
218,39.0,1.0,0.0,0.162559
234,33.0,1.0,0.0,0.014403
247,36.0,1.0,0.0,0.069651
